In [ ]:
import os
import csv
import concurrent.futures
from tqdm import tqdm

CHECKPOINT = "12-07_15-38"
PREFIX = "test"
MODEL_FOLDER = f"../ckpts/{CHECKPOINT}"
OUTPUT_FOLDER = f"../Predictions/predictions_{PREFIX}_{CHECKPOINT}"
Model_nums = [f[6:-4] for f in os.listdir(MODEL_FOLDER) if f.lower().endswith(('.pth'))]
Model_nums.sort()
print("Models:", Model_nums)

def test_model(epoch):
    print("Testing:", epoch)
    os.system(f"python3 test_v3.py --test_epochs {epoch} --model_folder {MODEL_FOLDER} --output_folder {OUTPUT_FOLDER} --prefix {PREFIX}")

# Set Max Threads

os.environ["OMP_NUM_THREADS"] = "24"

# Use concurrent.futures.ThreadPoolExecutor to run tests concurrently
with concurrent.futures.ThreadPoolExecutor() as executor:
    executor.map(test_model, Model_nums)


# Merge Parameters files
# Create a new CSV file to store the merged parameters
with open(f"{OUTPUT_FOLDER}/{CHECKPOINT}_{PREFIX}_all_miou.csv", "a") as f:
    csv_writer = csv.writer(f)
    csv_writer.writerow(["Epoch", "Mean_Iou", "Mean_Dice", "Mean_hd95"])

for epoch in tqdm(Model_nums):
    with open(f"{OUTPUT_FOLDER}/predictions_{epoch}/{epoch}_{PREFIX}_miou.csv", "r") as f:
        csv_reader = csv.reader(f)
        
        for row in csv_reader:
            if row[0] == "Average":
                with open(f"{OUTPUT_FOLDER}/{CHECKPOINT}_{PREFIX}_all_miou.csv", "a") as f:
                    csv_writer = csv.writer(f)
                    csv_writer.writerow([epoch, row[1], row[2], row[3]])

print("Merged CSV file created:", f"{OUTPUT_FOLDER}/{CHECKPOINT}_{PREFIX}_all_miou.csv")